# Personalising Spotify through a recommendation engine

## Imports

In [69]:
import pandas as pd
import numpy as np
import json
import re
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

import sqlite3

pd.options.mode.chained_assignment = None
#gets rid of annoying false-positive warning


In [70]:
%matplotlib inline
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [71]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [72]:
# try:
#     conn = sqlite3.connect("million_songs.db")
# except Exception as e:
#     print(e)

# # Now in order to read in pandas dataframe we need to know table name
# cursor = conn.cursor()
# songs_df = pd.read_sql_query('SELECT * FROM FEATURES_2', conn)
# many_genres_df = pd.read_sql_query('SELECT * FROM TRAINING_DB_2', conn)
# conn.close()

many_songs_df = pd.read_csv('data/songs.csv').head(100) #making it a small number so i can manage lol

In [73]:
with open('delete.json') as json_file:
    data = json.load(json_file)
test_df = pd.DataFrame(data['items'])

test_df



,added_at,is_local,track,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,2022-01-20T09:47:22Z,False,"{'album': {'images': [{'height': 640, 'url': '...",[lo-fi beats],0.795,0.327,4,-7.383,1,0.3050,0.88100,0.74300,0.1110,0.705,85.003,68SaMQq572M8M6etnXi5mM,171697,4
1,2022-01-20T09:49:36Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[chillhop, lo-fi beats, phonk, japanese chillh...",0.637,0.773,7,-8.710,1,0.0499,0.00481,0.25400,0.1540,0.680,147.996,6eLcyLIP80xK9wIa5dEJmq,129730,4
2,2022-01-20T09:55:03Z,False,"{'album': {'images': [{'height': 640, 'url': '...",[],0.677,0.376,6,-9.250,0,0.0400,0.69400,0.80500,0.0969,0.533,98.002,6shFdiaZFrhwDuYtweOtEY,93323,4
3,2022-01-20T09:57:59Z,False,"{'album': {'images': [{'height': 640, 'url': '...",[],0.486,0.559,10,-9.562,0,0.0557,0.14900,0.97000,0.1360,0.732,73.993,6zztFE3pi7i5QKZT3PIraN,161933,4
4,2022-01-20T10:01:40Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[chillhop, downtempo, japanese chillhop, japan...",0.681,0.676,2,-11.410,1,0.0370,0.72900,0.92400,0.1000,0.452,94.985,7KMm1zF2QOTSw2t01IAFif,247613,4
5,2022-01-20T10:04:45Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[chillhop, downtempo, japanese chillhop, japan...",0.625,0.910,7,-9.531,1,0.0345,0.02900,0.82500,0.0948,0.747,99.990,545g4jlvOqZo1sj6k1f9j9,440093,4
6,2022-01-20T10:04:58Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[chillhop, downtempo, japanese chillhop, japan...",0.673,0.509,1,-7.904,0,0.0365,0.27200,0.78500,0.1290,0.791,82.020,401EJO6SRKjlg5CZpo2SEl,220867,4
7,2022-01-20T10:12:41Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[charlotte nc indie, chillhop, wonky]",0.585,0.717,6,-6.171,0,0.0311,0.22400,0.74100,0.1330,0.754,83.989,35FrTrB3SRaSuz94x9SxRD,158929,4
8,2022-03-07T22:21:48Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[japanese chillhop, lo-fi beats, lo-fi chill, ...",0.505,0.355,7,-12.963,1,0.0920,0.47400,0.00471,0.0949,0.162,159.862,0CNuKKi6MHrr8DqYtpzbCD,210058,4
9,2022-03-07T22:22:32Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[chillhop, future bass, lo-fi beats, portland ...",0.680,0.414,2,-12.126,1,0.0957,0.91100,0.86800,0.4050,0.359,94.111,23mTYmF1nmxxTJoMjjIXzi,250213,4


In [74]:
# test_df = pd.get_dummies(test_df['genres'])
# test_df.reset_index(drop = True, inplace = True)



In [75]:
test_df.dtypes

added_at             object
is_local               bool
track                object
genres               object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
id                   object
duration_ms           int64
time_signature        int64
dtype: object

In [76]:
hmm_array = test_df.dtypes[test_df.dtypes == 'float64'].index.values
hmm_array = np.append(hmm_array, ['hell', 'yeah'])

hmm_array

array(['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'hell', 'yeah'], dtype=object)

In [77]:
many_songs_df['Genre'] = many_songs_df['Genre'].apply(lambda x: [x])
many_songs_df = many_songs_df.rename(str.lower, axis='columns')
many_songs_df['year'] = many_songs_df['release'].apply(lambda x: x.split('-')[0])
many_songs_df.head()

,name,id,artists,artists_id,release,duration,genre,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valeance,tempo,year
0,Don't Go Yet,1058fW9H3fZA6QjYCdOBad,['Camila Cabello'],['4nDoRrQiYLoBzwC5BhVJzF'],2021-07-23,164842,[pop],0.666,0.796,10,-6.967,0.1030,0.0492,0.000000,0.0442,0.6100,110.108,2021
1,What A Time (feat. Niall Horan),5WtfUKzXircvW8l5aaVZWT,"['Julia Michaels', 'Niall Horan']","['0ZED1XzwlLHW4ZaG4lOT6m', '1Hsdzj7Dlq2I7tHP75...",2019-01-24,173307,[pop],0.516,0.467,0,-6.184,0.0302,0.6620,0.000000,0.0853,0.3860,132.861,2019
2,Scars To Your Beautiful,42ydLwx4i5V49RXHOozJZq,['Alessia Cara'],['2wUjUUtkb5lvLKcGKsKqsR'],2016-03-11,230227,[pop],0.319,0.739,0,-5.740,0.2720,0.0285,0.000000,0.1110,0.4490,194.169,2016
3,Reckless,5ajjAnNRh8bxFvaVHzpPjh,['Madison Beer'],['2kRfqPViCqYdSGhYSM9R0Q'],2021-06-04,203385,[pop],0.386,0.426,3,-6.642,0.0363,0.8070,0.000000,0.1400,0.2610,180.104,2021
4,Dusk Till Dawn (feat. Sia) - Radio Edit,3e7sxremeOE3wTySiOhGiP,"['ZAYN', 'Sia']","['5ZsFI1h6hIdQRw2ti0hz81', '5WUlDfRSoLAfcVSX1W...",2018-12-14,239000,[pop],0.259,0.437,11,-6.589,0.0386,0.1020,0.000001,0.1060,0.0951,180.042,2018


## Create weight document for genres (one-time run)

In [78]:

def path_to_genre_weights(filepath):
	with open(filepath) as fp:
		all_genres = fp.read().split('\n')

	genre_range = len(all_genres)
	genre_step = 0.9/genre_range
	return np.arange(0.1, 1, genre_step), all_genres

filepath = 'data/genres.txt'
genre_weights, genre_names = path_to_genre_weights(filepath)
genre_weights_df = pd.DataFrame(genre_weights, columns=['weight'], index=genre_names)

genre_weights_df.to_csv('data/genre_weights.csv')
#saved for future use

In [79]:
genre_weights_df = pd.read_csv('data/genre_weights.csv', index_col=0)
#grabbin it again bc we need to use it. this code can be re-used lol
float_cols = many_songs_df.dtypes[many_songs_df.dtypes == 'float64'].index.values
int_cols = many_songs_df.dtypes[many_songs_df.dtypes == 'int64'].index.values
stat_cols = np.concatenate((float_cols, int_cols), axis=None)
#function to build entire feature set

def ohe_prep(df, column): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df



In [80]:


def create_feature_set(df, stat_cols, genre_weights_df):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        stat_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    # song_genre_weights_df = pd.DataFrame(
    #     [[0.0] * len(genre_weights_df.index)] * len(df.index), #make n (songs) rows x 5885 columns of 0's
    #     columns=genre_weights_df.index,
    # )

    # for index, song in df.iterrows():
    #     for genre in song['genre']:
    #         song_genre_weights_df.at[index, genre] = genre_weights_df['weight'][genre]

    song_genre_weights_df = (
        df['genre'].explode()
        .str.get_dummies().sum(level=0)
    )

    for genre in song_genre_weights_df.columns.values:
        song_genre_weights_df[genre] *= genre_weights_df.loc[genre].values[0]

    year_ohe = ohe_prep(df, 'year') * 0.5

    #scale float columns
    floats = df[stat_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([song_genre_weights_df, floats_scaled, year_ohe], axis = 1)
     
    #add song id
    final['id']=df['id'].values
    
    return final



In [81]:
feature_set = create_feature_set(many_songs_df, stat_cols, genre_weights_df)
feature_set.head()

C:\Users\ollie\AppData\Local\Temp\ipykernel_12184\3342468855.py:22: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['genre'].explode()


,pop,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valeance,tempo,duration,key,1999,2004,2005,2006,2007,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,id
0,0.1,0.116452,0.174485,0.129028,0.062152,0.010518,0.000000,0.002254,0.119565,0.061774,0.038016,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1058fW9H3fZA6QjYCdOBad
1,0.1,0.073534,0.093768,0.141452,0.003466,0.144941,0.000000,0.015304,0.068812,0.099188,0.049416,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,5WtfUKzXircvW8l5aaVZWT
2,0.1,0.017167,0.160500,0.148497,0.198388,0.005978,0.000000,0.023464,0.083086,0.200000,0.126073,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,42ydLwx4i5V49RXHOozJZq
3,0.1,0.036338,0.083710,0.134185,0.008384,0.176748,0.000000,0.032672,0.040489,0.176872,0.089924,0.054545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,5ajjAnNRh8bxFvaVHzpPjh
4,0.1,0.000000,0.086408,0.135026,0.010238,0.022100,0.000004,0.021876,0.002900,0.176770,0.137888,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,3e7sxremeOE3wTySiOhGiP


In [82]:
def summarise_playlist(playlist_df, weight_decay, not_playlist=False):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    if not_playlist:
        return playlist_df.sum(axis = 0).drop('id')
    else:
        playlist_feature_set = playlist_df.sort_values('added_at',ascending=False)
        most_recent_date = playlist_feature_set.iloc[0,-1]
        for ix, row in playlist_feature_set.iterrows():
            playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_decay ** (-x))
        playlist_feature_set_weighted = playlist_feature_set.copy()
        playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:].mul(playlist_feature_set_weighted.weight,0))
        playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :]
    
    return playlist_feature_set_weighted_final.sum(axis = 0)

In [83]:
def find_cosine_similarity(playlist_summary, wasabia_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_summary (pandas series): summarized playlist feature
        wasabia_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    wasabia_df = wasabia_features[['id']]
    wasabia_df['sim'] = cosine_similarity(wasabia_features.drop('id', axis = 1).values, playlist_summary.values.reshape(1, -1))[:,0]
    non_playlist_df_top_20 = wasabia_df.sort_values('sim',ascending = False).head(20)
    # non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_20

In [84]:
pretend_playlist = feature_set.head(40)
pretend_wasabia = feature_set.tail(40)

summarised_pretend_playlist = summarise_playlist(pretend_playlist, 1.09, not_playlist=True)

pretend_cosine_similarity = find_cosine_similarity(summarised_pretend_playlist, pretend_wasabia)

pretend_cosine_similarity

,id,sim
69,3P3pw6C19j31Rnzgo3JG7o,0.683641
62,4r1CoAATVuxag55Ct3Y1aX,0.677532
72,6KL88T4Ma4ABXqzgUoEwkd,0.649914
79,3hc7PLcmuZbJGPjhIore1i,0.637280
67,3je88Q4OvTqIx7BFRFYvRA,0.633145
77,7FGq80cy8juXBCD2nrqdWU,0.630500
66,2vXgyN14LX2zl7JEASw242,0.618916
70,0ZXdzaT1k688dkpNeEgQiV,0.617112
73,5qrSlOut2rNAWv3ubArkNy,0.602859
76,5VmpLtRycwbA54XsTffKq4,0.599007


Note to self:

Playlist -> Tracks 

-> Get features

-> Get artists -> Genres

Associate features + genres with track!!